In [ ]:
# Importar las bibliotecas necesarias
import pandas as pd
import numpy as np
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score
import matplotlib.pyplot as plt
import seaborn as sns

# Cargar los datos
hado_data = pd.read_csv('hado_data_clustering.csv')

# Mostrar las primeras filas del DataFrame
hado_data.head()

In [ ]:
hado_data.columns

In [ ]:
hado_data = hado_data.rename(columns={'diagnostico_cluster_optimal': 'diagnostico_kmeans',
                                      'motivo_ing_cluster_optimal': 'motivo_ing_kmeans',
                                      'motivo_alta_cluster_optimal': 'motivo_alta_kmeans',
                                      'diagnostico_topic': 'diagnostico_lda'
                                     }
                            )

Las métricas que utilizaré son:

1. **Índice de Silhouette**: Mide cuán similar es un objeto a su propio cluster en comparación con otros clusters. Los valores varían entre -1 y 1, donde un valor alto indica que el objeto está bien emparejado con su propio cluster y mal emparejado con los clusters vecinos.
  
2. **Índice de Calinski-Harabasz**: También conocido como el índice de varianza, es la relación entre la suma de las dispersiones dentro del cluster y la suma de las dispersiones entre los clusters. Los valores más altos indican mejores clusters.
  
3. **Índice de Davies-Bouldin**: Es la relación promedio de la similitud entre cada cluster y su cluster más similar. Los valores más bajos indican mejores clusters.

Voy a calcular estas métricas para cada método.

In [ ]:
hado_data = hado_data.dropna(subset=['diagnostico'])
hado_data.reset_index(drop=True, inplace=True)

In [ ]:
# Crear un objeto TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)

# Ajustar y transformar los datos de la columna 'diagnostico' al formato TF-IDF
diagnostico_tfidf = tfidf_vectorizer.fit_transform(hado_data['diagnostico'])

In [ ]:
# Suponiendo que las etiquetas de cluster para cada método se almacenan en las siguientes columnas:
# - 'diagnostico_kmeans' para K-means
# - 'diagnostico_lda' para LDA diagnostico_topic
# - 'diagnostico_hierarchical' para Clustering Jerárquico
# - 'diagnostico_dbscan' para DBSCAN

# Importar la biblioteca necesaria para TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

# Lista para almacenar los resultados
validation_metrics = []

# Calcular métricas de validación interna para cada método
for method, label_column in zip(['K-means', 'LDA', 'Hierarchical', 'DBSCAN'],
                                ['diagnostico_kmeans', 'diagnostico_lda', 'diagnostico_hierarchical', 'diagnostico_dbscan']):
    # Excluir clusters con etiqueta -1 (ruido en DBSCAN)
    valid_data = hado_data[hado_data[label_column] != -1]
    labels = valid_data[label_column]
    features = diagnostico_tfidf[valid_data.index]

    # Calcular métricas
    silhouette = silhouette_score(features, labels)
    calinski_harabasz = calinski_harabasz_score(features.toarray(), labels)
    davies_bouldin = davies_bouldin_score(features.toarray(), labels)

    # Almacenar resultados
    validation_metrics.append({'Method': method,
                               'Silhouette Score': silhouette,
                               'Calinski-Harabasz Score': calinski_harabasz,
                               'Davies-Bouldin Score': davies_bouldin})

# Convertir a DataFrame para facilitar la visualización
validation_metrics_df = pd.DataFrame(validation_metrics)
validation_metrics_df

Excelente, has calculado las métricas de validación interna para cada uno de los métodos de agrupamiento. Aquí tienes una breve interpretación de los resultados:

### Métricas de Validación Interna:

1. **Silhouette Score**: Cuanto más cercano esté el valor a 1, mejor es el agrupamiento. Un valor cercano a 0 indica que los clusters se superponen.
    - **DBSCAN** tiene el valor más alto (0.984), lo que sugiere que los clusters están bien separados.
    - **LDA** tiene el valor más bajo (0.0499), lo que podría indicar una superposición significativa entre los clusters.

2. **Calinski-Harabasz Score**: Cuanto más alto sea el valor, mejor es el agrupamiento.
    - **DBSCAN** también lidera aquí con un valor muy alto (5131.17).
    - **LDA** tiene el valor más bajo (67.37), lo que sugiere que podría no ser el mejor método para este conjunto de datos.

3. **Davies-Bouldin Score**: Cuanto más bajo sea el valor, mejor es el agrupamiento.
    - **DBSCAN** tiene el valor más bajo (0.241), lo que indica que los clusters están bien separados y tienen baja dispersión.
    - **LDA** tiene el valor más alto (5.717), lo que sugiere que los clusters podrían no estar bien definidos.

### Conclusión:

- **DBSCAN** parece ser el método más robusto para este conjunto de datos, según las métricas de validación interna.
- **LDA** parece ser el menos eficaz en términos de calidad del agrupamiento.

Estos resultados pueden ayudarte a decidir qué método de agrupamiento es el más adecuado para tu conjunto de datos y qué áreas podrían necesitar más investigación o ajuste.